In [ ]:
import numpy as np
import os
from scipy import ndarray

import skimage as sk
from skimage import transform
from skimage import util
from skimage import io
import imageio
import cv2

from numpy import linalg as LA
from scipy.sparse import csc_matrix,coo_matrix
from scipy.sparse.linalg import svds, eigs
from sklearn.decomposition import PCA,TruncatedSVD
import pickle
import pandas as pd
from random import randint
import random

import umap
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN, KMeans, AffinityPropagation, MeanShift
from sklearn.preprocessing import MinMaxScaler
import kmapper as km
from kmapper.cover import Cover
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
import hdbscan
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

import networkx as nx
from community import best_partition # this is not part of networkx

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from matplotlib.colors import ListedColormap
from scipy import ndimage
import imageio
import plotly
import plotly.graph_objs as go

def bbox(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]

    return rmin, rmax, cmin, cmax    
def embedding(data,dim):
    projection = mapper.fit_transform(data, projection=umap.UMAP(n_components=dim, n_neighbors=200, 
                                             a=None, angular_rp_forest=False, b=None, init='spectral',
                                           learning_rate=1.0, local_connectivity=1.0, metric='euclidean',
                                           metric_kwds=None, min_dist=0.1, n_epochs=500,
                                           negative_sample_rate=10, random_state=47,
                                           repulsion_strength=1.0, set_op_mix_ratio=0.5, spread=0.25,
                                           target_metric='categorical', target_metric_kwds=None,
                                           target_n_neighbors=-1, target_weight=0.5, transform_queue_size=10.0,
                                           transform_seed=42, verbose=False))
    return projection

def bbox(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]

    return rmin, rmax, cmin, cmax    

def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 0)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value

def rotate_image(mat, angle):
    """
    Rotates an image (angle in degrees) and expands image to avoid cropping
    """

    height, width = mat.shape[:2] # image shape has 3 dimensions
    image_center = (width/2, height/2) # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape

    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)

    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos = abs(rotation_mat[0,0]) 
    abs_sin = abs(rotation_mat[0,1])

    # find the new width and height bounds
    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_mat[0, 2] += bound_w/2 - image_center[0]
    rotation_mat[1, 2] += bound_h/2 - image_center[1]

    # rotate image with the new bounds and translated rotation matrix
    rotated_mat = cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h))
    return rotated_mat

import math

def round_up_to_even(f):
    return int(math.ceil(f / 2.) * 2)

In [ ]:
num_rnd_cell = 100
num_rnd_img = 30

In [ ]:
'''Loading the training data as a pandas dataframe'''
directories = ['/home/garner1/Work/dataset/cellImages/training_augmented/Cancer',
               '/home/garner1/Work/dataset/cellImages/training_augmented/Immuno',
              '/home/garner1/Work/dataset/cellImages/training_augmented/Other']

'''Box each nucleus'''
cell_labels = []
target_id = 0
widths = []
heights = []
target = []
for directory in directories:
    target_id += 1
    cell_id = 0
    cellList = random.sample(os.listdir(directory), k=num_rnd_cell)
    if directory == directories[0]: cellList_cancer = list(cellList)
    if directory == directories[1]: cellList_immuno = list(cellList)
    if directory == directories[2]: cellList_other = list(cellList)
    for cell in cellList:
        path = os.path.join(directory, cell)
        cell_id += 1
        imgList = random.sample(os.listdir(path), k=num_rnd_img) #sample images
        if directory == directories[0]: imgList_cancer = list(imgList)
        if directory == directories[1]: imgList_immuno = list(imgList)
        if directory == directories[2]: imgList_other = list(imgList)
        for img in imgList:
            filename = os.path.join(path, img)
            img = imageio.imread(filename)
            rmin, rmax, cmin, cmax = bbox(img)
            width = rmax-rmin
            height = cmax-cmin
            widths.append(width)
            heights.append(height)
            target.append(target_id)    
            cell_labels.append(cell_id)
        
            
'''Resizing images to small boxes'''
Mwidths = 60
Mheights = 60
images = []
for directory in directories:
    if directory == directories[0]: cellList = list(cellList_cancer)
    if directory == directories[1]: cellList = list(cellList_immuno)
    if directory == directories[2]: cellList = list(cellList_other)
    for cell in cellList:
        path = os.path.join(directory, cell)
        if directory == directories[0]: imgList = list(imgList_cancer)
        if directory == directories[1]: imgList = list(imgList_immuno)
        if directory == directories[2]: imgList = list(imgList_other)
        for img in imgList:
            filename = os.path.join(path, img)
            img = imageio.imread(filename)
            rmin, rmax, cmin, cmax = bbox(img)
            delta_w, delta_h = Mwidths-(rmax-rmin), Mheights-(cmax-cmin)
            top, bottom = delta_h//2, delta_h-(delta_h//2)
            left, right = delta_w//2, delta_w-(delta_w//2)
            newimg = np.pad(img[rmin:rmax,cmin:cmax],((left,right),(top,bottom)),'constant', constant_values=(0))
            images.append(newimg)
            
data = np.zeros((len(images),Mwidths*Mheights))
for ind in range(len(images)):
    data[ind,:] = images[ind].flatten() # from 2D arrays to 1D arrays

print(data.shape)

In [ ]:
'''3D embedding'''
n_neighbors = 30  #this parameter affects the distinguishability between clusters, the higher the better, but it costs computationally
%time embedding = umap.UMAP(n_neighbors=n_neighbors,min_dist=0.0,n_components=3,random_state=42).fit_transform(data) # do not include target together with data

In [ ]:
'''3D visualization of annotated data'''
# Configure Plotly to be rendered inline in the notebook.
plotly.offline.init_notebook_mode()

fig = go.Figure()
size = 10
names = ['cancer','immuno','other']
for label in set(target):
    labeled = [ind for ind in range(len(target)) if target[ind] == label]
    fig.add_trace(go.Scatter3d(
        x=embedding[labeled,0],  # <-- Put your data instead
        y=embedding[labeled,1],  # <-- Put your data instead
        z=embedding[labeled,2],  # <-- Put your data instead
        name=str(names[label-1]),
        mode="markers",
        marker=dict(color=label,size=2, opacity=0.5)
    ))
fig.update_layout(title_text="Reduction in 3D ",title_font_size=30)
fig.show()


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm, metrics
from sklearn.neighbors import KNeighborsClassifier

X_train, X_test, y_train, y_test = train_test_split(embedding,target,stratify=target,random_state=42)

# Create a classifier
# classifier = svm.SVC(gamma=0.001)
classifier = KNeighborsClassifier(n_jobs=-1)

# We learn the digits on the first half of the digits
classifier.fit(X_train, y_train)
predicted = classifier.predict(X_test)
print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(predicted, y_test)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(predicted, y_test))

In [ ]:
'''Select images where prediction is different from expert annotation'''
diff_index_list = [ind for ind in range(len(y_test)) if abs(predicted[ind]-y_test[ind])>0]

List = []
for j in diff_index_list:
    [List.append(index) for index in range(len(embedding)) if all(embedding[index]-X_test[j]==0)]

row_list = [List[ind] for ind in range(len(List)) if abs(predicted[ind]-y_test[ind])>0]
data_selection = data[row_list,:]
predicted_selection = [predicted[ind] for ind in range(len(List)) if abs(predicted[ind]-y_test[ind])>0]
y_test_selection = [y_test[ind] for ind in range(len(List)) if abs(predicted[ind]-y_test[ind])>0]

In [ ]:
print(__doc__)

sns.set(style='white', context='notebook', rc={'figure.figsize':(80,60)})

num_of_examples = 2
rnd = random.sample([ind for ind in range(len(predicted_selection))],k=num_of_examples)
predicted_images = list([zip(data_selection, predicted_selection)[ind] for ind in rnd])
for index, (image, label) in enumerate(predicted_images):
    plt.subplot(2, num_of_examples, index + 1)
    plt.axis('off')
    plt.imshow(image.reshape((Mwidths,Mheights)), cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('T: %i' % label,fontsize=100)
    

test_images_and_labels = list([zip(data_selection, y_test_selection)[ind] for ind in rnd])
for index, (image, label) in enumerate(test_images_and_labels):
    plt.subplot(2, num_of_examples, index + 1 + num_of_examples)
    plt.axis('off')
    plt.imshow(image.reshape((Mwidths,Mheights)), cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('P: %i' % label,fontsize=100)

plt.tight_layout(h_pad=0.5, w_pad=0.01)
plt.show()

Now we explore unsupervised clustering over the reduced data points. Reducing to 2d seems not to be sufficient to distinguish the immuno and cancer clusters. In 3d they become easily distinguishable.

In [ ]:
'''
HDBSCAN clusters in 3D
low min sample size seems to refuce unclustered data;
larger min cluster size decrease cluster numbers
PCA reduction might not be a good idea because shape space is non-linear and the linear reduction could distort distances and later clustering
'''
labels = hdbscan.HDBSCAN(min_samples=100,min_cluster_size=200).fit_predict(embedding)

import plotly.graph_objects as go

fig = go.Figure()
size = 1
for cluster in set(labels):
    clustered = (labels == cluster)
    fig.add_trace(go.Scatter3d(
        x=embedding[clustered,0],  # <-- Put your data instead
        y=embedding[clustered,1],  # <-- Put your data instead
        z=embedding[clustered,2],  # <-- Put your data instead
        name="cluster "+str(cluster),
        mode="markers",
        marker=dict(color=cluster+1,size=cluster+2, opacity=0.5)
    ))
fig.update_layout(title_text="HDBSCAN clusters in 3D ",title_font_size=30)
fig.show()
##########
clustered = (labels >= 0)
print('The percentage of clustered data points is '+str(np.sum(clustered) *1.0/ data.shape[0]*100)+'%')


In [ ]:
'''Classification report for unsupervised clustering'''
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm, metrics
from sklearn.neighbors import KNeighborsClassifier

X_train, X_test, y_train, y_test = train_test_split(embedding,labels,stratify=labels,random_state=42)

# Create a classifier
# classifier = svm.SVC(gamma=0.001)
classifier = KNeighborsClassifier(n_jobs=-1)

# We learn the digits on the first half of the digits
classifier.fit(X_train, y_train)
predicted = classifier.predict(X_test)
print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(predicted, y_test)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(predicted, y_test))

Predict unlabeled images

In [ ]:
'''
We can: 
1) first train the dimensionality reduction
2) load the unlabeld dataset
3) transform the unlabeled dataset
4) classify
5) Visualize
6) Map cluster label on the field of view
'''

'''_1_'''
n_neighbors = 30  #this parameter affects the distinguishability between clusters, the higher the better, but it costs computationally
%time trans = umap.UMAP(n_neighbors=n_neighbors,min_dist=0.0,n_components=3,random_state=42).fit(data) # do not include target together with data

In [ ]:
'''_2_'''
num_rnd_cell = 1699
num_rnd_img = 10
target = []
directory = '/home/garner1/Work/dataset/cellImages/image52/augmented'

cell_labels = []
images = []
cell_id = 0
count = 0
total = 0
imgLists = []
cellList = random.sample(os.listdir(directory), k=num_rnd_cell)
for cell in cellList:
    cell_id += 1
    path = os.path.join(directory, cell)
    imgList = random.sample(os.listdir(path), k=num_rnd_img) #sample images
    imgLists.append(imgList)
    for img in imgList:
        filename = os.path.join(path, img)
        img = imageio.imread(filename)
        rmin, rmax, cmin, cmax = bbox(img)
        delta_w, delta_h = Mwidths-(rmax-rmin), Mheights-(cmax-cmin)
        total += 1
        if delta_w >= 0:
            if delta_h >= 0:
                top, bottom = delta_h//2, delta_h-(delta_h//2)
                left, right = delta_w//2, delta_w-(delta_w//2)
                newimg = np.pad(img[rmin:rmax,cmin:cmax],((left,right),(top,bottom)),'constant', constant_values=(0))
                cell_labels.append(cell_id)
                images.append(newimg)
                
            
X_new = np.zeros((len(images),Mwidths*Mheights))
for ind in range(len(images)):
    X_new[ind,:] = images[ind].flatten() # from 2D arrays to 1D arrays

print(X_new.shape)

'''_3_'''
%time new_embedding = trans.transform(X_new)
'''_4_'''
new_predicted = classifier.predict(new_embedding)

In [ ]:
%time new_embedding = umap.UMAP(n_neighbors=n_neighbors,min_dist=0.0,n_components=3,random_state=42).fit_transform(X_new) # do not include target together with data
'''
HDBSCAN clusters in 3D
low min sample size seems to refuce unclustered data;
larger min cluster size decrease cluster numbers
PCA reduction might not be a good idea because shape space is non-linear and the linear reduction could distort distances and later clustering
'''
%time labels = hdbscan.HDBSCAN(min_samples=100,min_cluster_size=200).fit_predict(embedding)

import plotly.graph_objects as go

fig = go.Figure()
size = 1
for cluster in set(labels):
    clustered = (labels == cluster)
    fig.add_trace(go.Scatter3d(
        x=embedding[clustered,0],  # <-- Put your data instead
        y=embedding[clustered,1],  # <-- Put your data instead
        z=embedding[clustered,2],  # <-- Put your data instead
        name="cluster "+str(cluster),
        mode="markers",
        marker=dict(color=cluster+1,size=cluster+2, opacity=0.5)
    ))
fig.update_layout(title_text="HDBSCAN clusters in 3D ",title_font_size=30)
fig.show()
##########
clustered = (labels >= 0)
print('The percentage of clustered data points is '+str(np.sum(clustered) *1.0/ data.shape[0]*100)+'%')


In [ ]:
'''_5_'''
# Configure Plotly to be rendered inline in the notebook.
plotly.offline.init_notebook_mode()

fig = go.Figure()
for cluster in set(new_predicted):
    clustered = (new_predicted == cluster)
    fig.add_trace(go.Scatter3d(
        x=new_embedding[clustered,0],  # <-- Put your data instead
        y=new_embedding[clustered,1],  # <-- Put your data instead
        z=new_embedding[clustered,2],  # <-- Put your data instead
        name="cluster "+str(cluster),
        mode="markers",
        marker=dict(color=cluster+1,size=5, opacity=0.5)
    ))
fig.update_layout(title_text="Reduction in 3D ",title_font_size=30)
fig.show()

In [ ]:
import string 

def nochar(blabla):
    all = string.maketrans('','')
    nodigs = all.translate(all, string.digits)
    return blabla.translate(all, nodigs)

cell_ok = [-1]*len(cell_labels)
ind = 0
count_unique_cells = 0
image_ids = []
for cell in cellList:
    path = os.path.join(directory, cell)
    for img in imgLists[count_unique_cells]:
        filename = os.path.join(path, img)
        img = imageio.imread(filename)
        rmin, rmax, cmin, cmax = bbox(img)
        delta_w, delta_h = Mwidths-(rmax-rmin), Mheights-(cmax-cmin)
        image_ids.append(int(nochar(filename.split('/')[-2]))+1) #number cells adding 1 because id start from 0 on disk storage
        if delta_w >= 0:
            if delta_h >= 0:
                cell_ok[ind] = 1
        ind += 1
    count_unique_cells += 1

List = zip(image_ids,list(new_predicted))

from collections import defaultdict
d = defaultdict(list)

for k, v in List: 
    d[k].append(v) #group the cluster_id by cell_id

from collections import Counter
cluster_single_id = []
for List in d.values():
    c = Counter(List)
    cluster_single_id.append(c.most_common(1)[0][0]) #these are sorted by cell_id value from 1 to numb_new_cells

clustered = (np.asarray(cluster_single_id)>=0)

In [ ]:
import csv
with open('/home/garner1/Work/dataset/cellImages/image52/properties.csv', 'r') as f:
    properties = list(csv.reader(f, delimiter=','))
properties = np.array(properties)

fig = go.Figure()
for cluster in list(set(new_predicted))[:3]:
    clustered = (np.asarray(cluster_single_id)==cluster)
    cell_selection = np.asarray([d.keys()[ind] for ind in range(len(clustered)) if clustered[ind]])
    cluster_selection = np.asarray([d.values()[ind] for ind in range(len(clustered)) if clustered[ind]])
    fig.add_trace(go.Scatter(
        x=properties[cell_selection,1].astype(np.float),  # <-- Put your data instead
        y=properties[cell_selection,2].astype(np.float),  # <-- Put your data instead
        name="cluster "+str(cluster),
        mode="markers",
        marker=dict(color=cluster+1,size=10, opacity=0.5)
    ))
fig.update_layout(title_text="Reduction in 3D ",title_font_size=30,template="plotly_white")

fig.show()

In [ ]:
# image = imageio.imread('/home/garner1/Work/dataset/cellImages/image52/iMS266_20190426_001.sub52.jpg')
# sns.set(style='white', rc={'figure.figsize':(50,30)})
# plt.scatter(properties[np.asarray(d.keys())[clustered],1].astype(np.float),
#             properties[np.asarray(d.keys())[clustered],2].astype(np.float),
#             c=np.asarray(cluster_id)[clustered],
#             s=1000,
#             cmap='Spectral',
#             alpha=0.95);

# fig, ax = plt.subplots()
# for cluster in list(set(new_predicted))[:2]:
#     print cluster
#     clustered = (d.values() == cluster).flatten()
#     cell_selection = np.asarray([d.keys()[ind] for ind in range(len(clustered)) if clustered[ind]])
#     cluster_selection = np.asarray([d.values()[ind] for ind in range(len(clustered)) if clustered[ind]])
#     print len(cell_selection)
#     print len(cluster_selection.flatten())
#     ax.scatter(properties[cell_selection,1].astype(np.float),
#                 properties[cell_selection,2].astype(np.float),
#                 c=cluster_selection.flatten()+1,
#                 s=500,
#                 cmap='Spectral',
#                 alpha=0.75);
    
# plt.imshow(image,cmap='gray')
# plt.show()
